In [2]:
import gensim.models.doc2vec as d2v
import docIterators
import pymongo
from pymongo import MongoClient
import numpy as np

In [2]:
conn=['mongodb://localhost:6666','Cherry','Blueberry']
iterator = docIterators.BlueberryIter(conn,iter_type='LABELED_SENTENCES')

"model = d2v.Doc2Vec(size=100,window=5,min_count=1,alpha=0.025,min_alpha=0.025)\nmodel.build_vocab(iterator)\nfor epoch in range(10):\n    model.train(iterator)\n    model.alpha -=0.002\n    model.min_alpha=model.alpha\n    model.train(iterator)\nmodel.docvecs['10.4028/www.scientific.net/amr.488-489.1062']"

In [5]:
import pymongo
from pymongo import MongoClient
mc = MongoClient(conn[0])[conn[1]][conn[2]]

In [19]:
journals = []
ind=0
stupids=0
for record in mc.find():
    if 'journal' in record.keys():
        journals.append(record['journal'])
    else:
        stupids+=1
    ind+=1
    if ind%10000==0:
        print ind

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000


In [20]:
import collections
c = collections.Counter(journals)

In [28]:
testers =['Marine Chemistry',
'Atmospheric Environment',
'Supramolecular Chemistry',
'Journal of Atmospheric Chemistry']
marine_dois=[]
atmospheric_env_dois=[]
supramolecular_dois=[]
j_atmos_chem_dois=[]
for rec in mc.find({'journal':u'Marine Chemistry'}):
    marine_dois.append(rec['doi'])
for rec in mc.find({'journal':u'Atmospheric Environment'}):
    atmospheric_env_dois.append(rec['doi'])
for rec in mc.find({'journal':u'Supramolecular Chemistry'}):
    supramolecular_dois.append(rec['doi'])
for rec in mc.find({'journal':u'Journal of Atmospheric Chemistry'}):
    j_atmos_chem_dois.append(rec['doi'])

intra journal similarity

inter journal similarity

tsne clustering

comparisons between graphs

In [35]:
model = d2v.Doc2Vec.load('doc2vec')

In [49]:
def similarity(model,dois_1,dois_2=None):
    sim=0
    if dois_2:
        for i in range(len(dois_1)):
            for j in range(len(dois_2)):
                sim+=model.docvecs.similarity(dois_1[i],dois_2[j])
        sim=sim/(len(dois_1)*len(dois_2))
    else:
        for i in range(len(dois_1)):
            for j in range(i+1,len(dois_1)):
                sim+=model.docvecs.similarity(dois_1[i],dois_1[j])
        sim=sim/(len(dois_1)*(len(dois_1)-1)/2.)
    return sim

In [71]:
vec1=model.docvecs[marine_dois[0]]
vec2=model.docvecs[marine_dois[1]]
model.docvecs.most_similar([numpy.zeros(100)],topn=10)

[(u'10.1007/bf00471666', 0.0),
 (u'10.1007/bf00471668', 0.0),
 (u'10.1016/j.jfluchem.2004.09.009', 0.0),
 (u'10.1002/aoc.1518', 0.0),
 (u'10.1002/chem.200900660', 0.0),
 (u'10.1007/s00044-010-9317-2', 0.0),
 (u'10.1002/chin.200928052', 0.0),
 (u'10.1002/chem.200802451', 0.0),
 (u'10.1007/bf00471667', 0.0),
 (u'10.1002/ejoc.200900437', 0.0)]

In [62]:
print(mc.find_one({'doi':marine_dois[0]})[u'raw_title'])
for tup in model.docvecs.most_similar(marine_dois[0]):
    print('')
    print(mc.find_one({'doi':tup[0]})[u'raw_title'])

An automated procedure for laboratory and shipboard spectrophotometric measurements of seawater alkalinity: Continuously monitored single-step acid additions

Determination of titratable acidity and ascorbic acid in fruit juices in continuous-flow systems

Contributions of organic alkalinity to total alkalinity in coastal waters: A spectrophotometric approach

Shipboard Fluorometric Flow Analyzer for High-Resolution Underway Measurement of Ammonium in Seawater

Spectrophotometric determination of pH in seawater off Taiaroa Head, Otago, New Zealand: Full-spectrum modelling and prediction of pCO2 levels

Comparative study of chromatographic, spectrophotometric and non aqueous titrimetric methods for determination of protease inhibitor in tablets

Major Ion Chemistry and Sensitivity to Acid Precipitation of Sierra Nevada Lakes

FIA acid–base titrations with a new flow-through pH detector

A rapid, precise potentiometric determination of total alkalinity in seawater by a newly developed fl

In [51]:
testbed = {'Marine':marine_dois,
           'Atmospheric Env':atmospheric_env_dois,
           'Supramolecular':supramolecular_dois,
           'Journal Atmospheric':j_atmos_chem_dois}
for k,v in testbed.items():
    for k1,v1 in testbed.items():
        print(k + ' vs ' + k1)
        if k==k1:
            print(similarity(model,v))
        else:
            print(similarity(model,v,v1))

Journal Atmospheric vs Journal Atmospheric
0.225509094551
Journal Atmospheric vs Atmospheric Env
0.234635265326
Journal Atmospheric vs Marine
0.185787677823
Journal Atmospheric vs Supramolecular
-0.00928730320253
Atmospheric Env vs Journal Atmospheric
0.234635265326
Atmospheric Env vs Atmospheric Env
0.266419059019
Atmospheric Env vs Marine
0.198722768542
Atmospheric Env vs Supramolecular
-0.00735210834518
Marine vs Journal Atmospheric
0.185787677823
Marine vs Atmospheric Env
0.198722768542
Marine vs Marine
0.259197204374
Marine vs Supramolecular
-0.0117441583656
Supramolecular vs Journal Atmospheric
-0.00928730320253
Supramolecular vs Atmospheric Env
-0.00735210834518
Supramolecular vs Marine
-0.0117441583656
Supramolecular vs Supramolecular
0.078308480511


In [176]:
sample=[np.array([1,2,3]),np.array([1,2,3]),np.array([1,2,3])]
list(np.array(sample))

[array([1, 2, 3]), array([1, 2, 3]), array([1, 2, 3])]